In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
    ])

trainset = torchvision.datasets.ImageFolder(root='C:\\Users\\Sara Latif Khan\\OneDrive\\Desktop\\FYP_\\Scene15\\15-Scene\\train',transform=transform)
testset = torchvision.datasets.ImageFolder(root='C:\\Users\\Sara Latif Khan\\OneDrive\\Desktop\\FYP_\\Scene15\\15-Scene\\test', transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [4]:
for images, labels in trainloader:
    print(images.size(), labels.size())
    break

torch.Size([64, 3, 224, 224]) torch.Size([64])


In [5]:
model = models.vgg16(pretrained = True)

In [6]:
#First thing we want to do when treating our network as a fixed feature classifier is to freeze the network, so all we
#is to grab all the parameters and set the requires grad to false
#First thing we want to with our network to fine tune is to freeze earlier layers of our model

for param in model.parameters():  
    param.requires_grad = False

In [7]:
#Now let's train the fully connected head of our model, now let's take a look at our classiffier
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [8]:
#Now let's train the fully connected head of our model, so the first thing we'll do is to unfreeze this
for i in range(0,7):
    model.classifier[i].requires_grad = True

In [9]:
# Now we have an option of removing the classifier, now I'm gonna start training from the last layer, so let's just replace
# the fully connected layer with the/their 4096 features going to 1000 features
# So I can access this section by accessing the sixth row of the classifier or by an index of 6, and I'll add my own fully connected classifier
#that goes from a 4096 (features/ nodes) to 512 features/ nodes followed by a ReLU and dropout layer and then another fully connected
#going from 512 nodes to 15 output nodes/classes (bcoz I do have 15 classes in my dataset), and this is going to be followed by a logsoftmax

model.classifier[6] = nn.Sequential(
    nn.Linear(4096,512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512,15),
    nn.LogSoftmax( dim = 1 )
  )


In [10]:
model.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [11]:
criterion = nn.NLLLoss()
from torch.optim import Adam

model = model.to(device)  #Sending model to GPU
optimizer = Adam(model.parameters())  

# Training from the Fully Connected Section On Wards :

## Re-Training The Model :

In [13]:
# Training the model by Freezing everything except fully connected layers of the model

dataset = 'C:\\Users\\Sara Latif Khan\\OneDrive\\Desktop\\FYP_\\Scene15\\15-Scene'
model = model.to(device)
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

#Training Fixed Feature Extractor for 15 epochs
num_epochs = 5
batch_loss = 0
cum_epoch_loss = 0 #cumulative loss for each batch

for e in range(num_epochs):
    cum_epoch_loss = 0
  
    for batch, (images, labels) in enumerate(trainloader,1):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logps = model(images)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
    
        batch_loss += loss.item()
        print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')
        torch.save(model, dataset+'_model_'+str(e)+'.pt')
    
print(f'Training loss : {batch_loss/len(trainloader)}')  
    

Epoch(0/5 : Batch number(1/17) : Batch loss : 2.758256196975708


C:\Users\Sara Latif Khan\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type VGG. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Sara Latif Khan\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Sara Latif Khan\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Sara Latif Khan\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ".

Epoch(0/5 : Batch number(2/17) : Batch loss : 2.3670828342437744
Epoch(0/5 : Batch number(3/17) : Batch loss : 2.0973587036132812
Epoch(0/5 : Batch number(4/17) : Batch loss : 1.9258208274841309
Epoch(0/5 : Batch number(5/17) : Batch loss : 1.644967794418335
Epoch(0/5 : Batch number(6/17) : Batch loss : 1.5352137088775635
Epoch(0/5 : Batch number(7/17) : Batch loss : 1.3225994110107422
Epoch(0/5 : Batch number(8/17) : Batch loss : 1.1585873365402222
Epoch(0/5 : Batch number(9/17) : Batch loss : 0.8811482191085815
Epoch(0/5 : Batch number(10/17) : Batch loss : 0.9643864035606384
Epoch(0/5 : Batch number(11/17) : Batch loss : 0.7556966543197632
Epoch(0/5 : Batch number(12/17) : Batch loss : 0.7535868287086487
Epoch(0/5 : Batch number(13/17) : Batch loss : 0.5626626014709473
Epoch(0/5 : Batch number(14/17) : Batch loss : 0.5292341113090515
Epoch(0/5 : Batch number(15/17) : Batch loss : 0.7117160558700562
Epoch(0/5 : Batch number(16/17) : Batch loss : 0.6523106098175049
Epoch(0/5 : Batch n

# The Accuracy of Model

In [14]:
model. to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0
    
    #set_trace ()
    for batch, (images,labels) in enumerate(testloader,1):
        
        logps = model(images)
        output = torch.exp(logps)
        
        pred = torch.argmax(output,1)
        total += labels.size(0)
        
        num_correct += (pred==labels).sum().item()
        print(f'Batch ({batch} / {len(testloader)})')
        
        # to check the accuracy of model on 5 batches
        # if batch == 5:
            # break
            
    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total }% ')            
        
        
        

Batch (1 / 8)
Batch (2 / 8)
Batch (3 / 8)
Batch (4 / 8)
Batch (5 / 8)
Batch (6 / 8)
Batch (7 / 8)
Batch (8 / 8)
Accuracy of the model on 450 test images: 86.44444444444444% 


# Unfreezing and training over the last CNN block onwards 

## Re-training the model

In [15]:
# For Vgg-16 last block is 3 Convolution layers followed by a Max-Pooling Layer, so when we look at model we are looking 
# at features from index 24-31, so we are going to train for 5 epochs

for i in range(24,31):
    model.features[i].requires_grad = True


In [16]:
# Training the model again from the last CNN Block to The End of the Network
dataset = 'C:\\Users\\Sara Latif Khan\\OneDrive\\Desktop\\FYP_\\Scene15\\15-Scene'
model = model.to(device)
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

#Training Fixed Feature Extractor for 15 epochs
num_epochs = 5
batch_loss = 0
cum_epoch_loss = 0 #cumulative loss for each batch

for e in range(num_epochs):
    cum_epoch_loss = 0
  
    for batch, (images, labels) in enumerate(trainloader,1):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logps = model(images)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
    
        batch_loss += loss.item()
        print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')
        torch.save(model, dataset+'_model_'+str(e)+'.pt')
    
print(f'Training loss : {batch_loss/len(trainloader)}')  
    

Epoch(0/5 : Batch number(1/17) : Batch loss : 0.07259410619735718


C:\Users\Sara Latif Khan\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type VGG. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Sara Latif Khan\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Sara Latif Khan\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Sara Latif Khan\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ".

Epoch(0/5 : Batch number(2/17) : Batch loss : 0.23101936280727386
Epoch(0/5 : Batch number(3/17) : Batch loss : 0.06762712448835373
Epoch(0/5 : Batch number(4/17) : Batch loss : 0.12869267165660858
Epoch(0/5 : Batch number(5/17) : Batch loss : 0.3333587348461151
Epoch(0/5 : Batch number(6/17) : Batch loss : 0.05554826930165291
Epoch(0/5 : Batch number(7/17) : Batch loss : 0.20143495500087738
Epoch(0/5 : Batch number(8/17) : Batch loss : 0.07500186562538147
Epoch(0/5 : Batch number(9/17) : Batch loss : 0.09055822342634201
Epoch(0/5 : Batch number(10/17) : Batch loss : 0.21043643355369568
Epoch(0/5 : Batch number(11/17) : Batch loss : 0.1288479119539261
Epoch(0/5 : Batch number(12/17) : Batch loss : 0.1466018259525299
Epoch(0/5 : Batch number(13/17) : Batch loss : 0.08792094886302948
Epoch(0/5 : Batch number(14/17) : Batch loss : 0.10559666901826859
Epoch(0/5 : Batch number(15/17) : Batch loss : 0.16868874430656433
Epoch(0/5 : Batch number(16/17) : Batch loss : 0.11264611035585403
Epoch(

# The Accuracy of Model

In [17]:
model. to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0
    
    #set_trace ()
    for batch, (images,labels) in enumerate(testloader,1):
        
        logps = model(images)
        output = torch.exp(logps)
        
        pred = torch.argmax(output,1)
        total += labels.size(0)
        
        num_correct += (pred==labels).sum().item()
        print(f'Batch ({batch} / {len(testloader)})')
        
        # to check the accuracy of model on 5 batches
        # if batch == 5:
            # break
            
    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total }% ')            
        
        
        

Batch (1 / 8)
Batch (2 / 8)
Batch (3 / 8)
Batch (4 / 8)
Batch (5 / 8)
Batch (6 / 8)
Batch (7 / 8)
Batch (8 / 8)
Accuracy of the model on 450 test images: 86.88888888888889% 


# Unfreezing and training over the last two CNN block onwards

## Re-training the model 

In [18]:
#unfreezing and training the 2nd last CNN block
# For Vgg-16 last block is 3 Convolution layers followed by a Max-Pooling Layer, so when we look at model we are looking 
# at features from index 24-31, so we are going to train for 5 epochs

for i in range(17, 24):
    model.features[i].requires_grad = True


In [19]:
# Training the model again from the last CNN Block to The End of the Network
dataset = 'C:\\Users\\Sara Latif Khan\\OneDrive\\Desktop\\FYP_\\Scene15\\15-Scene'
model = model.to(device)
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

#Training Fixed Feature Extractor for 15 epochs
num_epochs = 5
batch_loss = 0
cum_epoch_loss = 0 #cumulative loss for each batch

for e in range(num_epochs):
    cum_epoch_loss = 0
  
    for batch, (images, labels) in enumerate(trainloader,1):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logps = model(images)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
    
        batch_loss += loss.item()
        print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')
        torch.save(model, dataset+'_model_'+str(e)+'.pt')
    
print(f'Training loss : {batch_loss/len(trainloader)}')  
    

Epoch(0/5 : Batch number(1/17) : Batch loss : 0.0014892679173499346


C:\Users\Sara Latif Khan\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type VGG. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Sara Latif Khan\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Sara Latif Khan\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Sara Latif Khan\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ".

Epoch(0/5 : Batch number(2/17) : Batch loss : 0.08129754662513733
Epoch(0/5 : Batch number(3/17) : Batch loss : 0.0035998360253870487
Epoch(0/5 : Batch number(4/17) : Batch loss : 0.008981808088719845
Epoch(0/5 : Batch number(5/17) : Batch loss : 0.026876050978899002
Epoch(0/5 : Batch number(6/17) : Batch loss : 0.005368266254663467
Epoch(0/5 : Batch number(7/17) : Batch loss : 0.03576454147696495
Epoch(0/5 : Batch number(8/17) : Batch loss : 0.008622827008366585
Epoch(0/5 : Batch number(9/17) : Batch loss : 0.0057098111137747765
Epoch(0/5 : Batch number(10/17) : Batch loss : 0.008979284204542637
Epoch(0/5 : Batch number(11/17) : Batch loss : 0.05422314256429672
Epoch(0/5 : Batch number(12/17) : Batch loss : 0.006226265802979469
Epoch(0/5 : Batch number(13/17) : Batch loss : 0.0029522578697651625
Epoch(0/5 : Batch number(14/17) : Batch loss : 0.01263727992773056
Epoch(0/5 : Batch number(15/17) : Batch loss : 0.0026471749879419804
Epoch(0/5 : Batch number(16/17) : Batch loss : 0.0130364

# The Accuracy of the model

In [20]:
model. to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0
    
    #set_trace ()
    for batch, (images,labels) in enumerate(testloader,1):
        
        logps = model(images)
        output = torch.exp(logps)
        
        pred = torch.argmax(output,1)
        total += labels.size(0)
        
        num_correct += (pred==labels).sum().item()
        print(f'Batch ({batch} / {len(testloader)})')
        
        # to check the accuracy of model on 5 batches
        # if batch == 5:
            # break
            
    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total }% ')            
        
        
        

Batch (1 / 8)
Batch (2 / 8)
Batch (3 / 8)
Batch (4 / 8)
Batch (5 / 8)
Batch (6 / 8)
Batch (7 / 8)
Batch (8 / 8)
Accuracy of the model on 450 test images: 85.11111111111111% 


In [12]:
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14')

In [14]:

model.eval()

from sklearn.metrics import confusion_matrix

nb_classes = 15

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

with torch.no_grad():
    for i, (inputs, classes) in enumerate(testloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

# Confusion matrix
conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())
print(conf_mat)

# Per-class accuracy
class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
print(class_accuracy)

[[ 0  4  0  0 20  0  0  3  0  0  0  2  0  0  1]
 [ 0  0  0  1 14  5  0  2  3  0  2  0  1  0  2]
 [ 0 10  0  0 10  0  0  6  0  0  0  1  1  1  1]
 [ 0  4  0  2 20  0  0  3  0  0  0  0  0  0  1]
 [ 0  2  0  1 15  2  0  5  0  0  0  1  0  0  4]
 [ 0  8  0  0 14  0  1  1  0  0  3  0  2  1  0]
 [ 0  9  0  0  6  0  0 14  0  0  0  1  0  0  0]
 [ 0  0  0  0 29  0  0  0  0  0  0  0  1  0  0]
 [ 0 12  0  2  8  1  0  1  0  0  0  3  3  0  0]
 [ 0  2  0  0 25  0  0  2  0  0  1  0  0  0  0]
 [ 0  5  0  2 10  0  0  7  0  0  5  0  0  1  0]
 [ 0  8  0  0 12  1  0  0  0  0  0  1  8  0  0]
 [ 0  9  0  1 11  3  0  4  0  0  0  0  0  0  2]
 [ 0 11  0  0 16  1  0  2  0  0  0  0  0  0  0]
 [ 0 18  0  0  6  0  0  4  0  0  0  0  0  0  2]]
[ 0.          0.          0.          6.66666667 50.          0.
  0.          0.          0.          0.         16.66666667  3.33333333
  0.          0.          6.66666667]
